---
title: NetOps
subject: Tutorial
subtitle: Notebook for UAV Cal/Val experiment in El Socorro
short_title: UAV Cal/Val experiment
authors:
  - name: Vicente Burchard-Levine
    affiliations:
    - Instituto de Ciencias Agrarias, ICA
    - CSIC
    orcid: 0000-0003-0222-8706
    email: vburchard@ica.csic.es
  - name: Juan José Peón
    affiliations:
    - Área de Sistemas de Teledetección
    - INTA
    orcid: 000
    email: jpeogar@inta.es
  - name: Héctor Nieto
    affiliations:
      - Instituto de Ciencias Agrarias, ICA
      - CSIC
    orcid: 0000-0003-4250-6424
    email: hector.nieto@ica.csic.es
  - name: Your names
    affiliations:
    - Your affiliation
    - Acronymn
    orcid: your orcid
---

# Summary
This notebook will perform a step-by-step guide on how to perform a calibration and valitation (Cal/Val) of different UAV-based sensors using vicarious in-situ measurements during UAV overpasses.

The data were collected on September 19th, 2022, at the El Socorro experimental vineyard farm managed by IMIDRA during PTI TELEDETECT workshop.


## Table of Contents
0. [Mount dataset from Google Drive](#connect-to-dataset-in-google-drive)
1. [Import Libraries](#import-libraries)
1. [Image selection](#select-and-image-to-work-with)
2. [Visualize and preprocess insitu ASD data](#preprocess-asd-data)
3. [Extracting UAV data over reference panels](#extract-uav-values-over-reference-panels)
4. [Develop empirical line](#develop-empirical-line)
5. [Calibrate and validate imagery based on empirical line](#Calibrate-image-with-developed-empirical-model)

> **Hint**: once each section is read, run the jupyter code cell underneath (marked as `[]`) by clicking the icon `Run`, or pressing the keys SHIFT+ENTER of your keyboard.


> **Note**: No programming skills are required to follow up this notebook. However, if you feel comfortable with Python, you are very welcome to edit the code and provide suggestions in this collaborative framework.


----

Code adapted from a [AET 2024](http://eo.csic.es/aet2024) course led by Juan José Peón | Área de Sistemas de Teledetección | Instituto Nacional de Técnica Aeroespacial (INTA) | jpeogar@inta.es

Original code available here: https://colab.research.google.com/drive/14GBdZ9rsC3nep4FKEVtSN-1VdcnB6kca

----

# Install libraries and dependencies

First step is to make sure you have all the dependencies. Refer to the readme file to install all dependencies. It also recommended to create a conda environment or make sure all libraries necessary are correctly installed.

## Installation

You should have these programs installed:

* Python and/or [Anaconda](https://www.anaconda.com/download/success). 

Install the requirements either with conda/mamba:

`mamba env create -f environment.yml`

or

`conda env create -f environment.yml`

or 

Make sure you have the required libraries installed:
- jupyter
- notebook
- jupyterlab
- jupyter-book
- gdal
- numpy
- scipy
- pandas
- matplotlib
- geopandas
- pathlib
- scikit-learn
- mystmd
- jupyterlab-myst


## Run the interactive book
Run the following command to initialize the book:

1. Activate python environment 

`conda activate calval-netops`

2. Initialize jupyter notebook
  
`jupyter notebook uav_calval_socorro.ipynb`

or 

`jupyter lab uav_calval_socorro.ipynb`



# Import Libraries	

In [ ]:
import json
import zipfile
import getpass
import numpy as np
import pandas as pd
import geopandas as gpd
from osgeo import gdal
from pathlib import Path
import matplotlib.pyplot as plt
from scipy.stats import linregress
from ipywidgets import interact, interactive, fixed, widgets
from IPython.display import display, Markdown, Latex
import os
import scipy.stats as st
import gc
import requests
import rasterstats
from rasterstats import zonal_stats

print('libraries imported correctly!')

## Dataset summary
A total of **10 UAV overpasses** were performed on September 19th, 2022, which included **6 multispectral** flights, **3 thermal infrared** flights, and **1 hyperspectral** flight.

See table below for a summary of UAV dataset acquired:

|Sensor|Sensor Type |Number of bands|Group |Flight Time (UTC)   
| :---     | :--- | :---  | :---  | :---  
| P4M      |Multi/VNIR | 5|ICMAN  | 08:43
| P4M      |Multi/VNIR | 5|ICMAN  | 09:14
| Sequoia+ |Multi/VNIR | 4|ICA  | 09:42
| Micasense|Multi/VNIR | 10|ICMAN  | 10:24
| Micasense|Multi/VNIR | 10|ICMAN  | 10:51
| P4M|Multi/VNIR | 5|GEOINCA  | 11:29
| H20T|TIR| 1|ICA  | 09:42
| M2EA|TIR| 1|ICMAN  | 11:30
| M2EA|TIR| 1|ICMAN  | 11:40
| Cubert S185|Hyper/VNIR| 125|EBD  | 11:57


# Select and image to work with

First we will select from {ref}`images-description` the images we will work with:

In [ ]:
w_image = widgets.Dropdown(
    options=[('P4M at 08:43', "P4M_ICMAN_0843"), ('P4M at 09:14', "P4M_ICMAN_0914"),
             ('Sequoia+ at 09:42', "Sequoia_ICA_0942"),
             ('Micasense at 09:58', "Micasense_ICMAN_0958"), ('Micasense at 10:49', "Micasense_ICMAN_1049"),
             ('P4M at 11:29', "P4M_GEOINCA_1129"), ('Cubert at 11:57', "Cubert_EBD_1157")
            ],
    value="Sequoia_ICA_0942",
    description='Image:',
)

display(Markdown("## Select an image"), w_image)

In [ ]:
#select orthomosaico or tiles
w_image_type = widgets.Dropdown(
    options=[('Orthomosaic', "Mosaic"), ('Individual Tile', "Panels")
            ],
    value="Mosaic",
    description='Image:',
)

display(Markdown("## Mosaic or tile?"), w_image_type)

## Set up working directory and get sensor metadata

In [ ]:
# set work directory (convert data directory as a Path object)
workdir = Path()

# set up out directory 
outdir = workdir / 'outputs'
# add dataset directory
datadir = workdir / 'dataset'
# We get the image information from the widget selection
sensor_name, sensor_group, sensor_time = w_image.value.split("_")

if sensor_name == 'Cubert':
  # folder with hyperspectral imagery
  uav_base_dir = datadir / 'dataset' / 'uav_imagery' / 'hyper'

else:
  # folder with multispetral imagery
  uav_base_dir = datadir / 'dataset' / 'uav_imagery' / 'multi'

if w_image_type.value == 'Panels':
  # directory to mosaic
  uav_dir = uav_base_dir / f"{sensor_name}_{sensor_group}" / 'tiles_panels'

  # get tile file list
  tile_files = list(uav_dir.glob(f'{w_image_type.value}_REF_{sensor_name}_{sensor_group}_*.tif'))

  # create option list based on available tiles
  option_list = []
  for file in tile_files:
    option_i = (file.name, file)
    option_list.append(option_i)


  #select orthomosaico or tiles
  w_tile_name = widgets.Dropdown(
      options=option_list,
      value=option_list[0][1],
      description='filename:',
  )

  #option_list
  display(Markdown("## Choose which tile to process"), w_tile_name)

else:
  # directory to mosaic
  uav_dir = uav_base_dir / f"{sensor_name}_{sensor_group}" / 'mosaic'


## Visualize UAV image
We will now visualize the image using false color image composite (R: NIR, G: Red, B: Green). For this, we incorporate the sensor description and metadata in a dictionary variable, including band center wavelength and width.


In [ ]:
# dictionary with relevant band info from each sensor
sensor_dict = {"Sequoia": {'center_wl':[550, 660, 735, 790],
                           'bandwidth':[40,40,10,40],
                           'false_color_bands':[4, 2, 1]}, # [nir, red, green]

               "Micasense":{'center_wl':[444, 475, 531, 560, 650, 668, 705, 717, 740, 842],
                            'bandwidth':[28, 32, 14, 27, 16, 14, 10, 12, 18, 57],
                            'false_color_bands':[10, 5, 4]}, # [nir, red, green]

               "P4M":{'center_wl':[450, 560, 650, 730, 840],
                      'bandwidth':[16, 16, 16, 16, 26],
                      'false_color_bands':[5, 3, 2]},

               "Cubert":{'center_wl':list(range(450, 947, 4)),
                         'bandwidth':[3.96344384, 4.22669042, 4.48993699, 4.74254005, 4.98598345, 5.22071147, 5.44653427, 5.66433633, 5.87442525, 6.07733135,
                                      6.27352687, 6.46250939, 6.64543238, 6.82258481, 6.99388625, 7.1595225, 7.32003116, 7.47551465, 7.62647149, 7.77271727,
                                      7.91450516, 8.19529191, 8.69417279, 9.17985398, 9.65092299, 10.108535, 10.5537396, 10.987298, 11.4081975, 11.8182273,
                                      12.2163699, 12.6047879, 12.9836208, 13.3515169, 13.710384, 14.0604483, 14.401957, 14.7351589, 15.0597232, 15.3761906,
                                      15.6847689, 15.9862039, 16.2807826, 16.5685624, 16.8492932, 17.1237615, 17.3924124, 17.6541616, 17.9099653, 18.1601686,
                                      18.4049445, 18.6445771, 18.8793717, 19.1088194, 19.3325072, 19.5520081, 19.7675729, 19.9782123, 20.1836628, 20.3856621,
                                      20.5839564, 20.7768683, 20.9669533, 21.1533516, 21.3350125, 21.5144747, 21.6891907, 21.8610244, 22.0300313, 22.1947156,
                                      22.3581132, 22.5158792, 22.6728688, 22.8250268, 22.9761498, 23.1228934, 23.2685793, 23.409865, 23.5505148, 23.686469,
                                      23.8224233, 23.9533522, 24.0840902, 24.2110763, 24.3366143, 24.460163, 24.5805574, 24.7009518, 24.8165964, 24.9320029,
                                      25.0453379, 25.1557019, 25.2660659, 25.3727363, 25.4782084, 25.5836805, 25.6843936, 25.7849903, 25.8852887, 25.9811129,
                                      26.076937, 26.1727435, 26.2638769, 26.3550104, 26.4461439, 26.5336399, 26.6201551, 26.7066702, 26.7913548, 26.8732907,
                                      26.9552267, 27.0371627, 27.1154286, 27.192903, 27.2703774, 27.3476769, 27.420713, 27.4937491, 27.5667852, 27.6397066,
                                      27.7083809, 27.7770553, 27.8457297, 27.9144041, 27.9830785],
                         'false_color_bands':[121, 51, 26]} # [nir, red, green]
              }

Now we will visualize the image as two panel figure. **Left panel:** mosaic and **Right panel:** zoomed over the reference panels


> **Note:** it might take a few moments for images to visualize, especially those with more bands


In [ ]:
# shapefile directory
vector_dir = datadir /'dataset'/ 'vector_data'

# panel outline shapefile
panels_file = vector_dir/'panels_borders.geojson'

# open as geopandas dataframe
panels_gdf = gpd.read_file(panels_file)

# sensor metadata
sensor_band_info = sensor_dict[sensor_name]

# plot for individual tile
if w_image_type.value == 'Panels':
  # directory to mosaic
  mosaic_file = w_tile_name.value
  filename = mosaic_file.name
  # update sensor time with tile time
  sensor_time = filename.split('_')[-2]

  # display image metadata
  display((Markdown(f"**Reading information from {sensor_name} sensor owned by {sensor_group}. Individual tile: {filename}**")))
  # open as GDAL object
  fid = gdal.Open(str(mosaic_file))

  # get number of bands
  num_bands = fid.RasterCount

  # get geotransform and other metadata
  geotransform = fid.GetGeoTransform()
  minx, maxy = geotransform[0], geotransform[3]
  maxx = minx + geotransform[1] * fid.RasterXSize
  miny = maxy + geotransform[5] * fid.RasterYSize

  # Read specific bands (4=NIR, 3=Red-Edge,2=Red and 1=Green)
  nir_band = sensor_band_info['false_color_bands'][0]
  red_band = sensor_band_info['false_color_bands'][1]
  green_band = sensor_band_info['false_color_bands'][2]

  if sensor_name == 'Cubert' or (sensor_name == 'P4M' and sensor_group=='ICMAN'):
    nir_ar = fid.GetRasterBand(nir_band).ReadAsArray()/10000  # Near-Infrared
    red_ar = fid.GetRasterBand(red_band).ReadAsArray()/10000  # Red
    green_ar = fid.GetRasterBand(green_band).ReadAsArray()/10000  # Green
  else:
    nir_ar = fid.GetRasterBand(nir_band).ReadAsArray()  # Near-Infrared
    red_ar = fid.GetRasterBand(red_band).ReadAsArray()  # Red
    green_ar = fid.GetRasterBand(green_band).ReadAsArray()  # Green

  # Stack the bands to obtain false colour image: R=NIR, G=Red, B=Green
  false_color_image = np.dstack((nir_ar, red_ar, green_ar))

  # delete array to avoid overuse of RAM
  del nir_ar, red_ar, green_ar
  del fid
  # make side by side false colour figure
  fig, ax = plt.subplots(1,1, figsize=(14,7))

  # whole study area
  ax.imshow(false_color_image, extent=[minx, maxx, miny, maxy])
  # overlay reference panel borders
  panels_gdf.plot(ax=ax, facecolor='none', edgecolor='indianred', linewidth=1)

  ax.set_title('False Colour (NIR, RED, GREEN): Study Site')
  # 4. Annotate the plot with labels from the GeoDataFrame
  for idx, row in panels_gdf.iterrows():
      # Get the centroid of the geometry (or use point coordinates directly)
      if row.geometry.geom_type == 'Point':
          x, y = row.geometry.x, row.geometry.y  # For points
      else:
          x, y = row.geometry.centroid.x, row.geometry.centroid.y  # For polygons

      if row['code'][:-1] == 'INTA':
          # Annotate the map with the 'name' field (or any other attribute)
          ax.annotate(text=row['code'], xy=(x, y),  xytext=(x, y+1.6), xycoords='data',
                      fontsize=5, ha='center', color='white')
      elif row['code'][:-1] == 'IAS':
          # Annotate the map with the 'name' field (or any other attribute)
          ax.annotate(text=row['code'], xy=(x, y),  xytext=(x, y+1.2), xycoords='data',
                      fontsize=6, ha='center', color='white')
      else:
          # Annotate the map with the 'name' field (or any other attribute)
          ax.annotate(text=row['code'], xy=(x, y),  xytext=(x, y+0.7), xycoords='data',
                      fontsize=6, ha='center', color='white')
  plt.show()

# plot for mosaic
else:
  # display image metadata
  display((Markdown(f"**Reading information from {sensor_name} sensor owned by {sensor_group} and acquired at {sensor_time[:2]}:{sensor_time[2:]}**")))

  # directory to mosaic
  uav_dir = uav_base_dir / f"{sensor_name}_{sensor_group}" / 'mosaic'

  # get orthomosaic file
  mosaic_file = uav_dir / f'Mosaic_REF_{w_image.value}.tif'

  # open as GDAL object
  fid = gdal.Open(str(mosaic_file))

  # get number of bands
  num_bands = fid.RasterCount

  # get geotransform and other metadata
  geotransform = fid.GetGeoTransform()
  minx, maxy = geotransform[0], geotransform[3]
  maxx = minx + geotransform[1] * fid.RasterXSize
  miny = maxy + geotransform[5] * fid.RasterYSize

  # Read specific bands (4=NIR, 3=Red-Edge,2=Red and 1=Green)
  nir_band = sensor_band_info['false_color_bands'][0]
  red_band = sensor_band_info['false_color_bands'][1]
  green_band = sensor_band_info['false_color_bands'][2]

  nir_ar = fid.GetRasterBand(nir_band).ReadAsArray()  # Near-Infrared
  red_ar = fid.GetRasterBand(red_band).ReadAsArray()  # Red
  green_ar = fid.GetRasterBand(green_band).ReadAsArray()  # Green

  # Stack the bands to obtain false colour image: R=NIR, G=Red, B=Green
  false_color_image = np.dstack((nir_ar, red_ar, green_ar))

  # delete array to avoid overuse of RAM
  del nir_ar, red_ar, green_ar
  del fid
  # make side by side false colour figure
  fig, axes = plt.subplots(1,2, figsize=(14,14))

  # whole study area
  ax1 = axes[0]
  ax1.imshow(false_color_image, extent=[minx, maxx, miny, maxy])
  # overlay reference panel borders
  panels_gdf.plot(ax=ax1, facecolor='none', edgecolor='indianred', linewidth=1)

  ax1.set_title('False Colour (NIR, RED, GREEN): Study Site')

  # zoom to reference panels
  ax2 = axes[1]
  ax2.imshow(false_color_image, extent=[minx, maxx, miny, maxy])
  # overlay reference panel borders
  panels_gdf.plot(ax=ax2, facecolor='none', edgecolor='indianred', linewidth=1)
  # 4. Annotate the plot with labels from the GeoDataFrame
  for idx, row in panels_gdf.iterrows():
      # Get the centroid of the geometry (or use point coordinates directly)
      if row.geometry.geom_type == 'Point':
          x, y = row.geometry.x, row.geometry.y  # For points
      else:
          x, y = row.geometry.centroid.x, row.geometry.centroid.y  # For polygons

      if row['code'][:-1] == 'INTA':
          # Annotate the map with the 'name' field (or any other attribute)
          ax2.annotate(text=row['code'], xy=(x, y),  xytext=(x, y+1.6), xycoords='data',
                      fontsize=5, ha='center', color='white')
      elif row['code'][:-1] == 'IAS':
          # Annotate the map with the 'name' field (or any other attribute)
          ax2.annotate(text=row['code'], xy=(x, y),  xytext=(x, y+1.2), xycoords='data',
                      fontsize=6, ha='center', color='white')
      else:
          # Annotate the map with the 'name' field (or any other attribute)
          ax2.annotate(text=row['code'], xy=(x, y),  xytext=(x, y+0.7), xycoords='data',
                      fontsize=6, ha='center', color='white')

  ax2.set_title('False Colour (NIR, RED, GREEN): Reference Panels')
  # set xy lim to zoom to panels, change for different zoom extent
  ax2.set_xlim(468115,468140)
  ax2.set_ylim(4442870,4442890)
  plt.show()
gc.collect()
del false_color_image

# Preprocess ASD data

ASD measurements were acquired over the references panels visible in the right panel of the above figure.

Three different rounds of measurements were acquired (R1, R2 and R3) over three different panel groups: **INTA, ICA and IAS**.

Each panel groups has different number of panels that roughly range from white to black.

**INTA** has *3 panels*, **ICA** has *7 panels* and **IAS** has *4 panels*.

In [ ]:
# get asd file directory
asd_file = datadir / 'dataset' / 'insitu' / 'ASDFieldSpec3_16120_SpecLab' / 'socorro_asd_panels_mean_all.csv'

# open asd file as a pandas directory
asd_df = pd.read_csv(asd_file)

print(f'Example of ASD dataset structure\n')
# show first five rows
print(asd_df.head())

# get wavelengths as series
wl_asd = asd_df["Wavelength"].values

# get id of all panels (omiting the first column which is the wavelength)
id_panel = asd_df.columns[1:]

# get panel ID to identify the panel type (i.e. INTA1, INTA2, INTA3 etc) i.e. remove round number 'R1', 'R2' etc
id_panel_type = id_panel.str.split('_').map(lambda x: x[0])
# get unique values of ID
panels_unique = id_panel_type.unique()

print(f'\nPanel groups and number:\n{list(panels_unique)}\n')


## Visualize ASD data

Plot ASD data from all rounds (R1, R2, and R3) for each group of panels (INTA, ICA, and IAS)

In [ ]:
# list of the panels from different groups
panel_groups = ['INTA', 'ICA', 'IAS']

# create 3x1 plot with each panel group
fig, axes = plt.subplots(3,1, figsize=(12,18))
i = 0
for group in panel_groups:
    # select all panels related to this group (INTA, ICA or IAS)
    groups_select = list(panels_unique[panels_unique.str.contains(group)])
    # sort from panel 1 to 4
    groups_select.sort()

    # initialize axis
    ax = axes[i]
    ax.set_title(f'{group} panels', fontsize=14)
    ax.set_xlabel('Wavelength (nm)', fontsize=12)
    ax.set_xlim(330, 2520)
    ax.set_ylabel('Reflectance (-)', fontsize=12)
    ax.set_ylim(0, 1.2)
    ax.grid(True)

    # create light to dark gradient depending on number of panels
    color_gradient = np.linspace(0.8, 0, len(groups_select)).astype(str)
    # plot for each panel number
    for n, g in enumerate(groups_select):
      cols = id_panel[id_panel.str.contains(g)]
      panel_number = asd_df[cols]
      # plot each of the spectra (each round)
      for col in panel_number.columns:
        ax.plot(wl_asd, panel_number[col].values, label=g, color=color_gradient[n])
    i = i + 1

# create legend for each subplot
for ax in axes:
    handles, labels = ax.get_legend_handles_labels()

    # Create a dictionary to remove duplicates (preserves the first occurrence)
    unique = dict(zip(labels, handles))

    # Add the legend with only the unique panel labels and color
    ax.legend(unique.values(), unique.keys(), loc='upper left', ncols = len(groups_select))

gc.collect()
plt.show()

## Convolve ASD spectra using the spectral response function (SRF)

We need to convolve the ASD measurements (400-2500nm) to the spectral response function (SRF) of the sensor. If SRF curve is available, the script will use that. If not, we assume a gaussian function using band center wavelength and bandwidth.

> **Note:** only Sequoia and Cubert have a SRF file available for the moment.

### Plot sensor spectral response function (SRF)

In [ ]:
# spectral response function (SRF) file
display(Markdown(f"**{sensor_name}_{sensor_group}**"))

# try to find SRF .txt file
srf_dir = uav_base_dir / f"{sensor_name}_{sensor_group}"
# looking for SRF in UAV base directory with certain format
srf_file_list = list(srf_dir.glob(f'SRF_{sensor_name}*.txt'))

# if SRF does not exists assume gaussian response over band center/width
if len(srf_file_list) == 0:
    # Define the Gaussian function
    def gaussian(x, A, mu, sigma):
        return A * np.exp(- (x - mu)**2 / (2 * sigma**2))

    # if SRF does not exists assume gaussian response over band center/width
    srf_dict = {'Wavelength_nm': np.arange(350,2501)}
    sensor_center_wl = sensor_dict[sensor_name]['center_wl']
    sensor_bandwidths = sensor_dict[sensor_name]['bandwidth']
    for n, band in enumerate(sensor_center_wl):
        # bandwidth, in this case, refers to the Full Width at Half Maximum (FWHM)
        FWHM = sensor_bandwidths[n]
        sigma = FWHM/(2*np.sqrt(2*np.log(2)))
        band_response = gaussian(srf_dict['Wavelength_nm'], 1, band, sigma)
        srf_dict[str(band)] = band_response

    srf_df = pd.DataFrame(srf_dict)

else:
    # use SRF file to plot curves
    srf_file = srf_file_list[0]
    # open as pandas dataframe
    srf_df = pd.read_csv(srf_file, sep = '\t')

band_names = srf_df.columns[1:]

# plot spectral response function
colormap = plt.cm.rainbow # Choose a colormap
colors = [colormap(x / (len(band_names) - 1)) for x in range(len(band_names))]

# Show spectral response function (SRF) curves
plt.figure(figsize=(9, 5))
plt.title(f'Spectral Response Function (SRF) - {sensor_name}', fontsize=14)
plt.xlabel('Wavelength (nm)', fontsize=12)
plt.xlim(400, 1000)
plt.ylabel('Relative Response (-)', fontsize=12)
plt.ylim(0, 1)
plt.grid(True)
i = 0
for band in band_names:
    plt.plot(srf_df['Wavelength_nm'], srf_df[band], color=colors[i], label = f'{str(band)}nm')
    i += 1
if sensor_name != 'Cubert':
  plt.legend(loc='upper left')
plt.show()


### Save convolved ASD data into dataframe

In [ ]:
# initialize empty dictionary to store results
asd_convolved_dict = {'panels': []}

# go through all panels (ICA1, ICA2 etc)
for panel in id_panel:
    asd_data = asd_df[panel]
    # add panel id to output dictonary
    asd_convolved_dict['panels'].append(panel)
    # for each band, apply SRF on ASD spectra
    for band in band_names:
        asd_band = np.sum(asd_data * srf_df[band])/np.sum(srf_df[band])
        # store result in output dictionary
        if band in asd_convolved_dict:
            asd_convolved_dict[band].append(asd_band)
        else:
            asd_convolved_dict[band] = []
            asd_convolved_dict[band].append(asd_band)

# convert output dictionary into pandas dataframe
asd_convolved_df = pd.DataFrame(asd_convolved_dict)
asd_convolved_df

## Plot convolved ASD data over reference panels

In [ ]:
# get center wl for each band
sensor_band_center = sensor_dict[sensor_name]['center_wl']

# create 3x1 plot with each panel group 
fig, axes = plt.subplots(3,1, figsize=(12,18))
i = 0
for group in panel_groups:
    
    # select all panels related to this group (INTA, ICA or IAS)
    groups_select = list(panels_unique[panels_unique.str.contains(group)])
    # sort from highest reflectance to lowest
    groups_select.sort()
    # intialize axis
    ax = axes[i]
    ax.set_title(f'{group} panels', fontsize=14)
    ax.set_xlabel('Wavelength (nm)', fontsize=12)
    ax.set_xlim(400, 900)
    ax.set_ylabel('Reflectance (-)', fontsize=12)
    ax.set_ylim(0, 1.2)
    ax.grid(True)
    
    # create light to dark gradient depending on number of panels
    color_gradient = np.linspace(0.8, 0, len(groups_select)).astype(str)
    
    # plot for each panel number
    for n, g in enumerate(groups_select):
      cols = id_panel[id_panel.str.contains(g)]
      panel_number = asd_df[cols]
        
      # plot each of the spectra (each round)
      for col in panel_number.columns:
        ax.plot(wl_asd, panel_number[col].values, label=g, color=color_gradient[n])

    # plot spectral response function
    colormap = plt.cm.rainbow # Choose a colormap
    colors = [colormap(x / (len(band_names) - 1)) for x in range(len(band_names))]
    n = 0
    for band in band_names:
        ax.plot(srf_df['Wavelength_nm'], srf_df[band], color=colors[n], label = f'{str(band)}nm')
        n = n + 1
    # get mean ASD values convolved to Sequoia bands
    for g in groups_select:
        srf_mask = asd_convolved_df['panels'].str.contains(g)
        convolved_values = asd_convolved_df[srf_mask]
        j = 0
        for band in band_names:
            asd_band_values = np.mean(convolved_values[band])
            ax.plot(sensor_band_center[j], asd_band_values, 'o', ms=6, c='black', mfc='deepskyblue')
            j = j +1
    
    i = i + 1
    
# create legend for each subplot
for ax in axes:
    handles, labels = ax.get_legend_handles_labels()
    
    # Create a dictionary to remove duplicates (preserves the first occurrence)
    unique = dict(zip(labels, handles))
    
    # Add the legend with only the unique panel labels and color
    ax.legend(unique.values(), unique.keys(), loc='upper left', ncols = len(groups_select))

plt.show()

# Extract UAV values over reference panels
Here, we will use the centroid of each panel shapefile and extract values over a radius of 25x25cm

:::{note}
Different approaches can be applied to extract UAV data over panels. This is just an example, we can discuss if we should test other approaches
:::



## Select which method you would use to extract the ASD information from the panels

In [ ]:
methods = [('Centroid point', 'centroid'),
           ('30cm buffer to centroid', 'centroid_buffer30cm'),
           ('Borders polygon', "borders")
         ]

w_extraction =widgets.Dropdown(
    options=methods,
    value="centroid",
    description='Extraction method:',
)

display(w_extraction)

### Visualize the geometry

In [ ]:
# shapefile directory
vector_dir = datadir / 'dataset' / 'vector_data'

print(f"Reading the geometry from {w_extraction.label}")
# panel outline shaefile
panels_file = vector_dir / f'panels_{w_extraction.value}.geojson'

# open panel point shapefile as geopandas dataframe
panels_gdf = gpd.read_file(panels_file)
panels_gdf

In [ ]:
# set up output folder
output_dir = Path(outdir) / f'{w_image.value}'

# check if it already exists, if not created
if not output_dir.exists():
  output_dir.mkdir()


no_data_value = 0
# Choose which zonal statistics you want to extract: "count min mean max median"
stat = "mean"
uav_values_df = pd.DataFrame(columns=["panel_id"])

for b in range(num_bands):
    # Define the name of the band
    col_id = f'B{b+1}_{stat}'
    # Extract zonal statistics using Raster Stats
    extract_json = zonal_stats(panels_gdf, mosaic_file, band_num=(b+1), stats=stat, geojson_out=True, nodata=no_data_value)
    # Loop along the panels extracted and create a table
    panel_values = {"panel_id": [], col_id:[]}
    for p in extract_json:
        panel_name = p["properties"]["code"]
        panel_values["panel_id"].append(panel_name)
        if (w_image_type.value == 'Panels' and (sensor_name == 'Cubert' or (sensor_name == 'P4M' and sensor_group =='ICMAN'))):
          panel_values[col_id].append(p["properties"][stat]/10000)
        else:
          panel_values[col_id].append(p["properties"][stat])

    panel_values = pd.DataFrame(panel_values)
    # Join tables
    uav_values_df = pd.merge(uav_values_df, panel_values, on="panel_id", how="outer")

# Saving extractions to a CSV file
csv_output = output_dir / f'{w_image_type.value}_REF_{sensor_name}_{sensor_group}_{sensor_time}_{w_extraction.value}.csv'
uav_values_df.to_csv(csv_output, index=False, sep=";")
print(f"Extractions saved in {csv_output}")

# Display table
uav_values_df

# Develop empirical line
## Plot UAV vs ASD reflectance over reference panel

:::{important}
For now only testing with ASD measurements from Round 1 (R1).
:::


## Calibrating image reflectance 

We will calibrate the UAV images using the empirical line approach using the reference panels.

:::{math}
\rho_{cal} = \mathbf{gain} (\rho_{original}) + \mathbf{offset}
:::

When we have three or more references (e.g. panels), then we perform the calibration by fitting a linear regression model
### Plot values over reference panels
Scatter plot showing **uncalibrated UAV reflectance** vs **ASD measurements** over reference panels.

In [ ]:
# generate color ramp based on number of bands in raster image
colormap = plt.cm.rainbow # Choose a colormap
colors = [colormap(x / (len(band_names) - 1)) for x in range(len(band_names))]

# create 1x3 plot with each panel group
fig, axes = plt.subplots(1,3, figsize=(14,4))
i = 0
for group in panel_groups:
    # select all panels related to this group (INTA, ICA or IAS)
    groups_select = list(panels_unique[panels_unique.str.contains(group)])
    groups_select.sort()

    # get asd data for particular group of pabels and round 1
    asd_data = asd_convolved_df[asd_convolved_df['panels'].str.contains(group) & asd_convolved_df['panels'].str.contains('R1')]
    # make sure panels sorted by lowest to highest
    asd_data = asd_data.sort_values(by = ['panels'])
    uav_data = uav_values_df[uav_values_df['panel_id'].str.contains(group)]
    # make sure panels sorted by lowest to highest
    uav_data = uav_data.sort_values(by = ['panel_id'])
    # intialize axis
    ax = axes[i]
    b = 0
    # plot for each band
    for band in asd_data.columns[1:]:
        asd_values = asd_data[band].values
        #if w_image_type.value == 'Panels' and sensor_name == 'Cubert':
        #  uav_values = uav_data[f'B{b+1}_{stat}'].values/10000
        #else:
        uav_values = uav_data[f'B{b+1}_{stat}'].values
        ax.scatter(uav_values, asd_values, color=colors[b],label = f'{band}nm', marker='o')
        b = b + 1

    ax.grid(True)
    ax.plot([0, 1.2], [0, 1.2], color='black', linestyle='--')
    # only add band legend if dealing with multispectral data
    if sensor_name != 'Cubert':
      ax.legend()
    # intialize axis
    ax = axes[i]
    ax.set_title(f'{group} panels', fontsize=14)
    ax.set_xlabel('UAV Reflectance (-)', fontsize=12)
    ax.set_xlim(0, 1.2)
    ax.set_ylabel('ASD Reflectance (-)', fontsize=12)
    ax.set_ylim(0, 1.2)
    ax.grid(True)
    i = i +1
plt.show()

In [ ]:
# use dictonary to store of empirical model parameters
lm_dict = {}

for group in panel_groups:
    # select all panels related to this group (INTA, ICA or IAS)
    groups_select = list(panels_unique[panels_unique.str.contains(group)])
    # get asd data for particular group of pabels and round 1
    asd_data = asd_convolved_df[asd_convolved_df['panels'].str.contains(group) & asd_convolved_df['panels'].str.contains('R1')]
    # make sure panels sorted by lowest to highest
    asd_data = asd_data.sort_values(by = ['panels'])
    uav_data = uav_values_df[uav_values_df['panel_id'].str.contains(group)]
    # make sure panels sorted by lowest to highest
    uav_data = uav_data.sort_values(by = ['panel_id'])

    # initialize empty dictionary for each panel group to store results
    lm_dict[group] = {}
    # go through each band and develop linear model
    b = 0
    for band in asd_data.columns[1:]:
        asd_values = asd_data[band].values
        #if w_image_type.value == 'Panels' and sensor_name == 'Cubert':
        #  uav_values = uav_data[f'B{b+1}_{stat}'].values/10000
        #else:
        uav_values = uav_data[f'B{b+1}_{stat}'].values
        nan_mask = np.logical_and(asd_values>0, uav_values>0)
        lm = linregress(uav_values[nan_mask], asd_values[nan_mask])
        gain = lm.slope
        offset = lm.intercept
        r2 = lm.rvalue ** 2
        # store result as list of [gain, offset, r2]
        lm_dict[group][band] = [gain, offset, r2]
        b = b + 1

el_output = {}
# convert dict to INTA panels
inta_df = pd.DataFrame(lm_dict['INTA'])
inta_df.index = ['gain', 'offset', 'R2']
# convert dict to ICA panels
ica_df = pd.DataFrame(lm_dict['ICA'])
ica_df.index = ['gain', 'offset', 'R2']
# convert dict to IAS panels
ias_df = pd.DataFrame(lm_dict['IAS'])
ias_df.index = ['gain', 'offset', 'R2']

# Save the empirical line models to a json
el_output["INTA"] = inta_df.to_dict()
el_output["ICA"] = ica_df.to_dict()
el_output["IAS"] = ias_df.to_dict()

# create output json file with fitted parameters
el_outputfile = output_dir / f'{w_image_type.value}_REF_{sensor_name}_{sensor_group}_{sensor_time}_EL_{w_extraction.value}.json'
with open(el_outputfile, "w") as outfile:
    json.dump(el_output, outfile, indent=4)

print(f"Saved Empirical-Line coefficients to {el_outputfile}")
pd.DataFrame(el_output)

### Visualize linear models

In [ ]:
colormap = plt.cm.rainbow # Choose a colormap
colors = [colormap(x / (len(band_names) - 1)) for x in range(len(band_names))]

# create 1x3 plot with each panel group
fig, axes = plt.subplots(1,3, figsize=(14,4))
i = 0
for group in panel_groups:
    # get linear model parameters associated to each group
    lm_panels = lm_dict[group]

    # select all panels related to this group (INTA, ICA or IAS)
    groups_select = list(panels_unique[panels_unique.str.contains(group)])
    # get asd data for particular group of panels and round 1
    asd_data = asd_convolved_df[asd_convolved_df['panels'].str.contains(group) & asd_convolved_df['panels'].str.contains('R1')]
    # make sure panels sorted by lowest to highest
    asd_data = asd_data.sort_values(by = ['panels'])
    uav_data = uav_values_df[uav_values_df['panel_id'].str.contains(group)]
    # make sure panels sorted by lowest to highest
    uav_data = uav_data.sort_values(by = ['panel_id'])
    # intialize axis
    ax = axes[i]
    b = 0
    for band in asd_data.columns[1:]:
        asd_values = asd_data[band].values
        #if w_image_type.value == 'Panels' and sensor_name == 'Cubert':
        #  uav_values = uav_data[f'B{b+1}_{stat}'].values/10000
        #else:
        uav_values = uav_data[f'B{b+1}_{stat}'].values
        # get lm params
        gain = lm_panels[band][0]
        offset = lm_panels[band][1]
        r2 = lm_panels[band][2]
        # plot linear regression
        x_lm = (np.nanmin(uav_values), np.nanmax(uav_values))
        y_lm = (gain*np.nanmin(uav_values) + offset, gain*np.nanmax(uav_values)+offset)
        ax.plot(x_lm, y_lm, color=colors[b], linestyle = '--')
        ax.scatter(uav_values, asd_values, color=colors[b],label = f'{band}nm, $R^2$ = {np.round(r2,3)}', marker='o')

        b = b + 1

    ax.grid(True)
    ax.plot([0, 1.2], [0, 1.2], color='black', linestyle='--')
    # only show legend if dealing with multispectral data
    if sensor_name != 'Cubert':
      ax.legend(fontsize=8)
    # intialize axis
    ax = axes[i]
    ax.set_title(f'{group} panels', fontsize=14)
    ax.set_xlabel('UAV Reflectance (-)', fontsize=12)
    ax.set_xlim(0, 1.2)
    ax.set_ylabel('ASD Reflectance (-)', fontsize=12)
    ax.set_ylim(0, 1.2)
    ax.grid(True)
    i = i +1
plt.show()

# Calibrate image with developed empirical model

Apply developed empirical line on each band of UAV image.
> **Note**: depending on image size and number of bands, this may take some time.

In [ ]:
# open uav image as gdal object
fid = gdal.Open(str(mosaic_file))

# get geotransform data
geotransform = fid.GetGeoTransform()
proj = fid.GetProjection()
minx, maxy = geotransform[0], geotransform[3]
maxx = minx + geotransform[1] * fid.RasterXSize
miny = maxy + geotransform[5] * fid.RasterYSize

# get metadata of image
num_bands = fid.RasterCount
template_ar = fid.GetRasterBand(1).ReadAsArray()
rows, cols = template_ar.shape

# initialize empty 3D array with same number of bands
ar_inta_cal = np.full((rows, cols, num_bands), np.nan)
ar_ica_cal = np.full((rows, cols, num_bands), np.nan)
ar_ias_cal = np.full((rows, cols, num_bands), np.nan)


for band in range(num_bands):
    print(f'- calibrating {sensor_name} Band {band+1}...')
    if (w_image_type.value == 'Panels' and (sensor_name == 'Cubert' or (sensor_name == 'P4M' and sensor_group =='ICMAN'))):
    #if w_image_type.value == 'Panels' and sensor_name == 'Cubert':
      ar = fid.GetRasterBand(band+1).ReadAsArray()/10000
    else:
      ar = fid.GetRasterBand(band+1).ReadAsArray()

    # get band name in string
    band_name = band_names[band]
    # inta LM/EL parameters
    gain, offset = lm_dict['INTA'][band_name][:2]
    # calibrate based on INTA reference panels
    ar_inta_cal[:,:,band] = (gain*ar) + offset
    # ica LM/EL parameters
    gain, offset = lm_dict['ICA'][band_name][:2]
    # calibrate based on ICA reference panels
    ar_ica_cal[:,:,band] = (gain*ar) + offset
    # ias LM/EL parameters
    gain, offset = lm_dict['IAS'][band_name][:2]
    # calibrate based on IAS reference panels
    ar_ias_cal[:,:,band] = (gain*ar) + offset

# save calibrated images as geotiffs
for group in panel_groups:
    outfilename = f'{w_image_type.value}_REF_{sensor_name}_{sensor_group}_{sensor_time}_EL_{group}panels_{w_extraction.value}.tif'
    outfile = output_dir / outfilename
    driver = gdal.GetDriverByName('GTiff')
    nbands = ar_inta_cal.shape[2]
    print(f'- saving image calibrated with {group} panels...')
    ds = driver.Create(str(outfile), cols, rows, nbands, gdal.GDT_Float32)
    ds.SetGeoTransform(geotransform)
    ds.SetProjection(proj)
    input_nodata = np.nan
    if group == 'INTA':
        ar_img = ar_inta_cal
    elif group == 'ICA':
        ar_img = ar_ica_cal
    else:
        ar_img = ar_ias_cal

    for i in range(nbands):
        band = ds.GetRasterBand(i+1)
        band.SetNoDataValue(input_nodata)
        array = ar_img[:,:,i]
        array[np.isnan(array)] = input_nodata
        band.WriteArray(array)
        band.FlushCache()

    ds.FlushCache()
    ds = None
    print(f'Calibrated image using {group} panels saved as {str(outfile)}')
gc.collect()
del fid
del ar_inta_cal, ar_ica_cal, ar_ias_cal

## Evaluate calibrated images against ASD data 
### extract calibrated UAV data over reference panels

In [ ]:
uav_cal_values_df = pd.DataFrame(columns=["panel_id"])

for group in panel_groups:
    # get calibrated images
    img_file = output_dir / f'{w_image_type.value}_REF_{sensor_name}_{sensor_group}_{sensor_time}_EL_{group}panels_{w_extraction.value}.tif'
    for b in range(num_bands):
        # Define the name of the band
        col_id = f'B{b+1}_{group}'
        print(f'Band {b+1} {group} panels')
        # Extract zonal statistics using Raster Stats
        extract_json = zonal_stats(panels_gdf, img_file, band_num=(b+1), stats=stat, geojson_out=True)
        # Loop along the panels extracted and create a table
        panel_values = {"panel_id": [], col_id:[]}
        for p in extract_json:
            panel_name = p["properties"]["code"]
            panel_values["panel_id"].append(panel_name)
            panel_values[col_id].append(p["properties"][stat])

        panel_values = pd.DataFrame(panel_values)
        # Join tables
        uav_cal_values_df = pd.merge(uav_cal_values_df, panel_values, on="panel_id", how="outer")

# Save the extractions to a CSV table
csv_output = output_dir / f'{w_image_type}_REF_{sensor_name}_{sensor_group}_{sensor_time}_EL_{w_extraction.value}.csv'
uav_cal_values_df.to_csv(csv_output, index=False, sep=";")
print(f"Empirical-line Extractions saved to {csv_output}")

# Display the extractions
uav_cal_values_df

### Plot calibrated UAV reflectance against ASD data

Compare calibrated UAV reflectance against ASD measurements. We will also save error metrics (RMSE, R2, bias) into a csv file.


In [ ]:
# function to calculate model performance metrics
def error_metrics(X, Y):
    rmse = np.sqrt(np.nanmean((X - Y) ** 2))
    cor = st.pearsonr(X[np.logical_and(~np.isnan(X),~np.isnan(Y))],
                      Y[np.logical_and(~np.isnan(Y),~np.isnan(X))])[0]
    r2 = cor ** 2
    bias = np.nanmean(X - Y)
    return rmse, r2, bias

# define colormap depending on number of bands
colormap = plt.cm.rainbow # Choose a colormap
colors = [colormap(x / (len(band_names) - 1)) for x in range(len(band_names))]

# initialize output dataframe
results_df = pd.DataFrame(columns=["bands"])


# create 1x3 plot with each panel group
fig, axes = plt.subplots(3,2, figsize=(14,18))
i = 0
for group in panel_groups:
    # select all panels related to this group (INTA, ICA or IAS)
    groups_select = list(panels_unique[panels_unique.str.contains(group)])
    # get asd data for particular group of pabels and round 1
    asd_data = asd_convolved_df[asd_convolved_df['panels'].str.contains(group) & asd_convolved_df['panels'].str.contains('R1')]
    asd_data = asd_data.sort_values(by = ['panels'])
    # original UAV data over panels
    uav_data = uav_values_df[uav_values_df['panel_id'].str.contains(group)]
    uav_data = uav_data.sort_values(by = ['panel_id'])

    # calibrated UAV data over panels
    cal_data = uav_cal_values_df[uav_cal_values_df['panel_id'].str.contains(group)]
    cal_data = cal_data.sort_values(by = ['panel_id'])

    # intialize axis
    ax = axes[i,0]
    b = 0
    rmse_og = []

    # output columns
    rmse_id = f'orig_{group}_rmse'
    r2_id = f'orig_{group}_R2'
    bias_id = f'orig_{group}_bias'
    # initialize output dict
    band_metrics = {"bands": [], rmse_id:[], r2_id:[], bias_id:[]}

    for band in asd_data.columns[1:]:
        band_metrics['bands'].append(band)
        asd_values = asd_data[band].values
        uav_values = uav_data[f'B{b+1}_{stat}'].values
        nan_mask = np.logical_and(asd_values > 0, uav_values > 0)
        rmse, r2, bias = error_metrics(asd_values[nan_mask], uav_values[nan_mask])
        band_metrics[rmse_id].append(rmse)
        band_metrics[r2_id].append(r2)
        band_metrics[bias_id].append(bias)
        rmse_og.append(rmse)
        ax.scatter(uav_values, asd_values, color=colors[b],label = f'{band}nm, rmse = {np.round(rmse,3)}', marker='o')
        b = b + 1
    # get mean rmse for all bands
    rmse_og_mean = np.nanmean(rmse_og)
    band_metrics_df = pd.DataFrame(band_metrics)
    # Join tables
    results_df = pd.merge(results_df, band_metrics_df, on="bands", how="outer")

    ax.grid(True)
    ax.plot([0, 1.2], [0, 1.2], color='black', linestyle='--')
    ax.text(0.75, 0.7, f'RMSE(mean) = {np.round(rmse_og_mean,4)}', weight='bold')

    if sensor_name != 'Cubert':
      ax.legend()
    ax.set_title(f'{group} panels - Original', fontsize=14)
    ax.set_xlabel('UAV Reflectance (-)', fontsize=12)
    ax.set_xlim(0, 1.2)
    ax.set_ylabel('ASD Reflectance (-)', fontsize=12)
    ax.set_ylim(0, 1.2)
    ax.grid(True)

    ax2 = axes[i,1]
    b = 0
    rmse_cal = []

    # output column
    rmse_id = f'EL_{group}_rmse'
    r2_id = f'EL_{group}_R2'
    bias_id = f'EL_{group}_bias'

    band_metrics = {"bands": [], rmse_id:[], r2_id:[], bias_id:[]}
    for band in asd_data.columns[1:]:
        band_metrics['bands'].append(band)
        asd_values = asd_data[band].values
        cal_values = cal_data[f'B{b+1}_{group}'].values
        rmse, r2, bias = error_metrics(asd_values, cal_values)
        band_metrics[rmse_id].append(rmse)
        band_metrics[r2_id].append(r2)
        band_metrics[bias_id].append(bias)
        rmse_cal.append(rmse)
        ax2.scatter(cal_values, asd_values, color=colors[b],label = f'{band}nm, rmse = {np.round(rmse,3)}', marker='o')

        b = b + 1
    # get mean rmse for all bands
    rmse_cal_mean = np.nanmean(rmse_cal)
    band_metrics_df = pd.DataFrame(band_metrics)
    # Join tables
    results_df = pd.merge(results_df, band_metrics_df, on="bands", how="outer")

    ax2.grid(True)
    ax2.plot([-0.2, 1.2], [-0.2, 1.2], color='black', linestyle='--')
    ax2.text(0.75, 0.7, f'RMSE(mean) = {np.round(rmse_cal_mean,4)}', weight='bold')
    if sensor_name != 'Cubert':
      ax2.legend()
    ax2.set_title(f'{group} panels - Calibrated', fontsize=14)
    ax2.set_xlabel('UAV Reflectance (-)', fontsize=12)
    ax2.set_xlim(0, 1.2)
    ax2.set_ylabel('ASD Reflectance (-)', fontsize=12)
    ax2.set_ylim(0, 1.2)
    ax2.grid(True)
    i = i +1
plt.show()
# Save the model metric results to a CSV table
csv_output = output_dir / f'Results_{sensor_name}_{sensor_group}_{sensor_time}_REF_{w_extraction.value}.csv'
results_df.to_csv(csv_output, index=False, sep=";")
print(f"Model metric results saved to {csv_output}")
results_df

# Next Steps?

- evaluate different sensor types (TIR)
- individual tile vs orthomosaic
- homogeneity analysis of panels
- best method to extract UAV data over panels? Radius (square, circle)? what size?
- timing of ASD measurements (round 1 vs round 2 vs round 3)
- number of panels to use?
- other??